
This notebook trains a logistic regression model based on fairness constraints outlined in this paper: https://arxiv.org/pdf/1507.05259.pdf on COMPAS data, using race as the sensitive attribute

In [8]:
import numpy as np
from numpy.core.fromnumeric import transpose
import pandas as pd

In [9]:
df = pd.read_csv('COMPAS_preprocessed.csv')
df.head(5)

,Unnamed: 0,age,c_charge_degree,race,score_text,sex,priors_count,days_b_screening_arrest,decile_score,is_recid,two_year_recid,c_jail_in,c_jail_out,days_in_jail
0,1,34,0,1,-1,1,0,-1.0,3,1,1,2013-01-26 03:45:27,2013-02-05 05:36:53,10
1,2,24,0,1,-1,1,4,-1.0,4,1,1,2013-04-13 04:58:34,2013-04-14 07:02:04,1
2,6,41,0,0,0,1,14,-1.0,6,1,1,2014-02-18 05:08:24,2014-02-24 12:18:30,6
3,8,39,1,0,-1,0,0,-1.0,1,0,0,2014-03-15 05:35:34,2014-03-18 04:28:46,2
4,10,27,0,0,-1,1,0,-1.0,4,0,0,2013-11-25 06:31:06,2013-11-26 08:26:57,1


In [10]:
cols =  ['age', 'c_charge_degree', 'race', 'score_text', 'sex',
       'priors_count', 'days_b_screening_arrest', 'decile_score', 'is_recid',
       'two_year_recid', 'days_in_jail']
df = df[cols]
df.head(5)

,age,c_charge_degree,race,score_text,sex,priors_count,days_b_screening_arrest,decile_score,is_recid,two_year_recid,days_in_jail
0,34,0,1,-1,1,0,-1.0,3,1,1,10
1,24,0,1,-1,1,4,-1.0,4,1,1,1
2,41,0,0,0,1,14,-1.0,6,1,1,6
3,39,1,0,-1,0,0,-1.0,1,0,0,2
4,27,0,0,-1,1,0,-1.0,4,0,0,1


In [11]:
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
col_X = ['age', 'c_charge_degree', 'score_text', 'sex',
       'priors_count', 'days_b_screening_arrest', 'decile_score', 'is_recid', 
        'days_in_jail']
X = df[col_X]
Z = df['race']
y = df['two_year_recid']
X_train, X_test, Z_train, Z_test, y_train, y_test = train_test_split( X, Z, y, test_size=0.33, random_state=42)

We've imported our dataset, and we only select the columns we need, as well as our target variable: two_year_recid Feature seelction is based on this notebook: https://github.com/propublica/compas-analysis/blob/master/Compas%20Analysis.ipynb

In [12]:
scaler_train = preprocessing.StandardScaler().fit(X_train)
scaler_test = preprocessing.StandardScaler().fit(X_test)

We use scikit learn's framework for optimization: https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.minimize.html https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.minimize.html#scipy.optimize.minimize https://scipy-lectures.org/advanced/mathematical_optimization/auto_examples/plot_non_bounds_constraints.html

Each of the constraints required for the framework to work needs to be programmed s.t. >= 0 for it to be true.

In [13]:
import scipy
from scipy.optimize import minimize
from numpy import linalg as LA

k = X_train.shape[1]
N = X_train.shape[0]

#equation 9 from paper
def upper_theta_constraint(params, X, Z, c, k):
    theta = params[:k]
    return ((-1/len(Z)) * np.matmul(np.matmul(transpose(Z - Z.mean()), X), theta)) + c

def lower_theta_constraint(params, X, Z, c, k):
    theta = params[:k]
    return ((np.matmul(np.matmul(transpose(Z - Z.mean()), X), theta))/len(Z)) + c  

def svm_loss(params, X, y, C, k):
    theta = params[:k]
    phi = params[k:]
    y_hat = y * np.dot(X,theta) 
    y_hat = np.maximum(np.zeros_like(y_hat), (1-y_hat)) 
    
    return C*sum(y_hat)

def phi_constraint(params,k):
  #theta = params[:k]
    phi = params[k:]

    return phi 

def phi_constraint2(params, k, y , X):
    theta = params[:k]
    phi = params[k:]
    return np.dot( transpose(theta), np.matmul(y,X)) - 1 + sum(phi)

In [15]:
theta = np.array(np.random.uniform(size=k)).reshape(-1, 1)
phi = np.array(np.random.uniform(size=N))
params = np.append(theta.flatten(), phi.flatten())
X_train_scaled = scaler_train.transform(X_train)
res = scipy.optimize.minimize(svm_loss, x0=params, args=(X_train_scaled, y_train, 0.8, k), 
                        method='SLSQP', 
                        constraints=({'type': 'ineq', 'fun': upper_theta_constraint, 'args': (X_train_scaled, Z_train , 0.8, k)},
                                     {'type': 'ineq', 'fun': lower_theta_constraint, 'args': (X_train_scaled, Z_train , 0.8, k)},
                                      {'type': 'ineq', 'fun': phi_constraint, 'args': [k]}),
                                     {'type': 'ineq', 'fun': phi_constraint2, 
                                      'args': [k, y_train, scaler_train.transform(X_train)]})

In [16]:
pd.DataFrame(res.x).to_csv('svm_parameters_second_try.csv')

In [17]:
import scipy
from scipy.optimize import minimize
from numpy import linalg as LA

k = X_train_scaled.shape[1]
N = X_train_scaled.shape[0]

params_hat = pd.read_csv('svm_parameters_second_try.csv')
theta_hat = params_hat['0'][0:k]

Now we evaluate for our evaluation metrics on our test accordingly.

In [18]:
X_test_scaled = scaler_test.transform(X_test)
yhat_test = np.matmul(theta_hat, np.transpose(X_test_scaled))

In [25]:
yhat_test[yhat_test < 0]

array([ -6.5787291 , -10.12326661, -16.50723231,  -7.52874247,
        -7.62988383,  -6.87659367,  -6.66070049,  -7.11501846,
        -7.44795851,  -8.88717776, -10.42619581,  -8.90393294,
        -8.13645599,  -7.25185987,  -8.80720499, -12.44389505,
       -10.04748646,  -5.45245145,  -6.65051443,  -6.67769703,
       -12.82139246,  -7.87537439, -16.22477462,  -8.83853321,
        -8.54653109, -16.92249005, -12.45017494, -12.53546802,
        -8.66603163,  -9.81351443,  -8.83557382,  -9.13068031,
        -7.37873104,  -5.37474662,  -6.68319096,  -7.27974719,
       -10.82229821, -11.17543547,  -9.82559656, -13.37912781,
       -13.31470705,  -5.94089805,  -8.13668801,  -4.28211558,
       -12.10243837,  -4.95190303,  -8.27407644,  -7.34286958,
       -15.93315028,  -7.98531434,  -6.32677443, -11.74444881,
       -12.83645286,  -8.2957357 ,  -7.50436028, -10.14669704,
        -8.35880584,  -7.01923537,  -9.70536395,  -7.67351646,
        -6.70274536, -16.41843452,  -9.1698855 ,  -4.37

# eval metric 1: Accuracy
Now we evaluate for accuracy

In [26]:

len(yhat_test[((yhat_test > 0) & (y_test > 0)) | ((yhat_test <= 0) & (y_test <= 0))])/len(yhat_test)

0.9735935706084959

# eval metric 2: Calibration 
Now we evaluate for calibration

In [30]:
odds_pos = yhat_test[Z_test == 1]
odds_neg = yhat_test[Z_test == 0]
y_pos = y_test[Z_test == 1]
y_neg = y_test[Z_test == 0]

calibration_pos = len(odds_pos[((odds_pos > 0) & (y_pos > 0)) |
    ((odds_pos <= 0) & (y_pos <= 0))])/len(odds_pos)
calibration_neg = len(odds_neg[((odds_neg > 0) & (y_neg > 0)) |
    ((odds_neg <= 0) & (y_neg <= 0))])/len(odds_neg)
calibration_pos, calibration_neg   

(0.9727443609022557, 0.9749262536873157)

The sensitive group is 97% calibrated and the the nonprotected group is 97.5% accurate, meaning the model is able to accurately identify both groups

# eval metric 3: equality of odds

In [29]:
#protected: y = 1, then y =0 
print(len(y_pos[(odds_pos > 0) & (y_pos > 0)]) / len(y_pos[y_pos > 0]), len(y_pos[(odds_pos > 0) & (y_pos <= 0)]) / len(y_pos[y_pos <= 0]))
#nonprotected: y = 1, then y =0 
print(len(y_neg[(odds_neg > 0) & (y_neg > 0)]) / len(y_neg[y_neg > 0]), len(y_neg[(odds_neg > 0) & (y_neg <= 0)]) / len(y_neg[y_neg <= 0]))

1.0 0.055876685934489405
1.0 0.04009433962264151


The model is able to predict both groups within the postive target variable 100% of the time, and the negative target 5.6% for the protected group and 4% for the non protected group

# eval metric 4: parity

In [33]:
(len(odds_neg[odds_neg > 0])/len(odds_neg),
  len(odds_pos[odds_pos > 0])/len(odds_pos))

(0.3997050147492625, 0.5394736842105263)

The model predict that 40% of the nonprotected group will return to criminal behaviour and 54% of the protected will do so as well - this is close to the true values; 37.4% and 51.2% respectively

In [34]:
(len(y_neg[y_neg > 0])/len(y_neg),
  len(y_pos[y_pos > 0])/len(y_pos))

(0.3746312684365782, 0.5122180451127819)